### **Deep Learning - Maestría Managment & Analytics**

## Transcripción de texto con GPT5

En esta notebook vamos a probar la api de GPT5 para leer un documento y transcribirlo. Para ello, nos vamos a conectar a la API. En mi caso, cree un entorno nuevo para evitar conflictos entre librerías, especialmente opencv. 
Importante: Tenés que configurar la API Key en tu entorno para correr este script.

1. Instalamos librerías necesarias

In [ ]:
import os
import io
from pathlib import Path
from time import sleep
from typing import List, Tuple

import numpy as np
import cv2
import fitz  # PyMuPDF
from PIL import Image


In [1]:
import sys, cv2, fitz, PIL, numpy, openai
print(sys.executable)
print("OpenCV:", cv2.__version__)


d:\conda_envs\gpt5-ocr\python.exe
OpenCV: 4.12.0


2. Definimos las rutas y modelo

In [ ]:
INPUT_DIR = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\CDs_Ejemplo"
OUTPUT_BASE = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo"
OUTPUT_DIR = os.path.join(OUTPUT_BASE, "Extraccion_GPT5")
TMP_DIR = os.path.join(OUTPUT_BASE, "_tmp_gpt5_preproc")

# Modelo
MODEL = "gpt-5"           
MAX_OUTPUT_TOKENS = 16000 

3. Definición del prompt de transcripción.

Estuvimos haciendo varias pruebas por lo que definimos un flag "Best-Guess". Cuando está activado (True) si un carácter o palabra es dudoso, infiere lo más probable según el contexto y marcá el fragmento con (?). Esto puede apagarse en caso de que consigamos documentos con mejor escaneo o resolución.

In [ ]:
BEST_GUESS = True

def build_prompt(best_guess: bool) -> str:
    if best_guess:
        return (
            "Transcribí el texto EXACTO de este documento.\n"
            "- No reformules ni resumas.\n"
            "- Si un carácter o palabra es dudoso, INFERÍ lo más probable según el contexto y marcá el fragmento con (?) de forma mínima (ej: 'Rivadavia(?) 1234').\n"
            "- Conservá saltos de línea, mayúsculas, puntuación y orden.\n"
            "- No agregues comentarios ni metadatos."
        )
    return (
        "Transcribí el texto EXACTO de este documento.\n"
        "- No reformules ni resumas.\n"
        "- Conservá saltos de línea, mayúsculas, puntuación y orden.\n"
        "- No agregues comentarios ni metadatos.\n"
    )

PROMPT = build_prompt(BEST_GUESS)

También definimos un prompt para minimizar posibles rechazos por copyright

In [ ]:
SYSTEM_MSG = (
    "El usuario proporcionó el archivo adjunto y autoriza su transcripción exacta para uso propio. "
    "La transcripción está permitida por ser contenido suministrado por el usuario. "
    "No rechaces por derechos de autor."
)

MIN_REASONABLE_LEN = 80
REFUSAL_SNIPPETS = (
    "no puedo ayudar", "i can't help", "cannot help", "no puedo", "rechazar",
)


4. Definimos algunos parámetros de preprocesamiento de la imagen para facilitar su lectura. Estos parámetros están explicados en detalle en el anexo final.

In [ ]:
USE_PREPROCESSING = True  

RENDER = dict(
    dpi=350,         
    grayscale=True,  # render en escala de grises (menos ruido y tamaño)
)

PREPROC = dict(
    denoise_h=7,           # 0 desactiva; 5–12 reduce grano (fastNlMeans)
    clahe_clip=2.0,        # la imagen se divide en una rejilla de mosaicos (tiles y el contraste se ajusta independientemente en cada mosaico.
    tile_grid=8,           # tamaño de bloque para CLAHE
    sharpen_amount=0.6,    # controla cuánto “enfoque” extra se aplica a la imagen después de limpiarla y mejorar el contraste.
    threshold="adaptive",  # Define cómo se binariza la imagen (pasar de escala de grises a blanco/negro) durante el preprocesamiento. L
    deskew=True,           # corregir inclinación leve
)

KEEP_TEMP_FILES = False    

5. Conexión a api key de open ai

In [ ]:
try:
    from openai import OpenAI
except ImportError:
    raise SystemExit("Falta 'openai'. Instalá con: pip install --upgrade openai")

client = OpenAI()  # toma OPENAI_API_KEY del entorno

6. Funciones para trabajar con los archivos

In [ ]:
def ensure_dir(path: str) -> None:
    Path(path).mkdir(parents=True, exist_ok=True)

def list_pdfs(folder: str) -> List[Path]:
    p = Path(folder)
    return sorted([f for f in p.iterdir() if f.suffix.lower() == ".pdf" and f.is_file()])

def already_done(out_txt: Path) -> bool:
    return out_txt.exists() and out_txt.stat().st_size > 0

def safe_write_text(path: Path, text: str) -> None:
    path.write_text(text, encoding="utf-8", errors="ignore")

Convertimos cada pdf a imagen para procesar con OpenCV

In [ ]:
def render_pdf_to_images(pdf_path: Path, dpi: int = 350, grayscale: bool = True) -> List[np.ndarray]:
    doc = fitz.open(pdf_path)
    scale = dpi / 72.0
    mat = fitz.Matrix(scale, scale)
    imgs = []
    for page in doc:
        pix = page.get_pixmap(matrix=mat, alpha=False)
        img_bytes = pix.tobytes("png")
        pil = Image.open(io.BytesIO(img_bytes))
        if grayscale:
            pil = pil.convert("L")  # 8-bit gray
        arr = np.array(pil)
        if arr.ndim == 2:
            pass
        else:
            arr = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
        imgs.append(arr)
    doc.close()
    return imgs

Ahora vamos a definir algunas funciones para pre- procesar. Entre ellas:
- Función de aplicación de **CLAHE (Contrast Limited Adaptive Histogram Equalization)**. Mejora el contraste de manera local dividiendo la imagen en mosaicos de tile × tile píxeles.
- Función de aplicación de **unsharp masking**: Desenfoca la imagen (GaussianBlur), resta el desenfoque de la imagen original para resaltar bordes y luego mezcla resultado con la original usando amount. Esto sirve para aumentar la nitidez de bordes de las letras, quedan más definidas.
- Función de **threshold**: Convierte a blanco/negro según el método adaptativo. Esto es  binarización local por bloques (mejor en iluminación irregular). Así se elimina fondo y realza texto, pero puede perder detalles finos si es muy agresivo.
- Función **deskew**: Corrige inclinación del texto.
- **preprocess_image** es el pipeline que orquesta todo




In [10]:
def _apply_clahe(gray: np.ndarray, clip: float, tile: int) -> np.ndarray:
    clahe = cv2.createCLAHE(clipLimit=clip, tileGridSize=(tile, tile))
    return clahe.apply(gray)

def _unsharp(gray: np.ndarray, amount: float) -> np.ndarray:
    if amount <= 0:
        return gray
    blur = cv2.GaussianBlur(gray, (0, 0), 1.0)
    sharp = cv2.addWeighted(gray, 1 + amount, blur, -amount, 0)
    return sharp

def _threshold(gray: np.ndarray, mode: str) -> np.ndarray:
    if mode == "none":
        return gray
    if mode == "otsu":
        _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        return bw
    if mode == "adaptive":
        bw = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 31, 10)
        return bw
    return gray

def _deskew(binary_or_gray: np.ndarray) -> np.ndarray:
    if binary_or_gray.ndim == 3:
        gray = cv2.cvtColor(binary_or_gray, cv2.COLOR_BGR2GRAY)
    else:
        gray = binary_or_gray.copy()
    _, bw = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    inv = cv2.bitwise_not(bw)
    coords = cv2.findNonZero(inv)
    if coords is None or len(coords) < 50:
        return binary_or_gray
    rect = cv2.minAreaRect(coords)
    angle = rect[-1]
    if angle < -45:
        angle = -(90 + angle)
    else:
        angle = -angle
    if abs(angle) < 0.3 or abs(angle) > 15:
        return binary_or_gray
    (h, w) = gray.shape[:2]
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle, 1.0)
    rotated = cv2.warpAffine(binary_or_gray, M, (w, h), flags=cv2.INTER_LINEAR, borderMode=cv2.BORDER_REPLICATE)
    return rotated

def preprocess_image(img: np.ndarray,
                     denoise_h: int = 7,
                     clahe_clip: float = 2.0,
                     tile_grid: int = 8,
                     sharpen_amount: float = 0.6,
                     threshold: str = "adaptive",
                     deskew: bool = True) -> np.ndarray:
    if img.ndim == 3:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        gray = img
    if denoise_h and denoise_h > 0:
        gray = cv2.fastNlMeansDenoising(gray, None, h=denoise_h, templateWindowSize=7, searchWindowSize=21)
    gray = _apply_clahe(gray, clip=clahe_clip, tile=tile_grid)
    gray = _unsharp(gray, amount=sharpen_amount)
    out = _threshold(gray, mode=threshold)
    if deskew:
        out = _deskew(out)
    return out

NameError: name 'np' is not defined

En este bloque vamos a definir una serie de funciones que sirven para crear un auto-selector de preprocesamiento. Esto sirve para crear varias versiones del documento, evaluar enfoque, contraste y detalle. Nos quedamos con la imagen más prometedora para OCR. Detalle de las funciones:

- Quality_scores(gray_or_bin): Calcula métricas de calidad visual de una imagen.

        - lap_var: varianza del Laplaciano → mide el nivel de nitidez. Cuanto más alto, más enfocada está.
        - contrast: desvío estándar de los píxeles → mide el contraste global.
        - edges: aplica Canny edge detection para detectar bordes.
        - edge_density: proporción de píxeles que son bordes → mide nivel de detalle.

- Score_for_selection(img): Combina las métricas en un único puntaje.
- Generate_variants(img): Genera tres versiones distintas de la misma imagen para luego elegir la mejor.
- Pick_best_variant(variants): Evalúa cada variante con _score_for_selection y elige la de mayor puntaje.

In [ ]:
def _quality_scores(gray_or_bin: np.ndarray) -> dict:
    if gray_or_bin.ndim == 3:
        gray = cv2.cvtColor(gray_or_bin, cv2.COLOR_BGR2GRAY)
    else:
        gray = gray_or_bin
    lap_var = cv2.Laplacian(gray, cv2.CV_64F).var()          # enfoque
    contrast = float(gray.std())                             # contraste global
    edges = cv2.Canny(gray, 50, 150)
    edge_density = float(np.count_nonzero(edges)) / edges.size
    return {"lap_var": lap_var, "contrast": contrast, "edge_density": edge_density}

def _score_for_selection(img: np.ndarray) -> float:
    q = _quality_scores(img)
    return q["lap_var"] + 40.0 * q["edge_density"] + 0.5 * q["contrast"]

def generate_variants(img: np.ndarray) -> list:
    base = preprocess_image(
        img,
        denoise_h=int(PREPROC.get("denoise_h", 7)),
        clahe_clip=float(PREPROC.get("clahe_clip", 2.0)),
        tile_grid=int(PREPROC.get("tile_grid", 8)),
        sharpen_amount=float(PREPROC.get("sharpen_amount", 0.6)),
        threshold=str(PREPROC.get("threshold", "adaptive")),
        deskew=bool(PREPROC.get("deskew", True)),
    )
    hi_contrast = preprocess_image(
        img,
        denoise_h=max(5, int(PREPROC.get("denoise_h", 7) - 2)),
        clahe_clip=3.0,
        tile_grid=8,
        sharpen_amount=0.5,
        threshold="otsu",
        deskew=True,
    )
    if img.ndim == 3:
        g = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    else:
        g = img
    up = cv2.resize(g, None, fx=2.0, fy=2.0, interpolation=cv2.INTER_LANCZOS4)
    up = _rl_deblur(up, iterations=4)
    up = _apply_clahe(up, clip=2.5, tile=8)
    up = _unsharp(up, amount=0.7)
    up = _threshold(up, mode="adaptive")
    up = _deskew(up)
    return [base, hi_contrast, up]

def pick_best_variant(variants: list) -> np.ndarray:
    scores = [_score_for_selection(v) for v in variants]
    return variants[int(np.argmax(scores))]

Definimos una función para convertir la lista de imágenes (ya procesadas en memoria como numpy.ndarray) en un único archivo PDF.

In [ ]:
def images_to_pdf(images: List[np.ndarray], out_pdf_path: Path, dpi: int = 350) -> None:
    pil_imgs = []
    for im in images:
        if im.ndim == 2:
            pil = Image.fromarray(im)  # 'L'
        else:
            pil = Image.fromarray(cv2.cvtColor(im, cv2.COLOR_BGR2RGB))
        pil_imgs.append(pil.convert("L"))
    pil_imgs[0].save(out_pdf_path, "PDF", resolution=dpi, save_all=True, append_images=pil_imgs[1:])

Nos conectamos con la API de GPT para enviar el PDF y recibir la transcripción.

In [ ]:
def upload_pdf(pdf_path: Path):
    return client.files.create(file=pdf_path.open("rb"), purpose="user_data")

def transcribe_pdf_fileid(file_id: str) -> str:
    # Primer intento
    resp = client.responses.create(
        model=MODEL,
        max_output_tokens=MAX_OUTPUT_TOKENS,
        input=[
            {"role": "system", "content": [{"type": "input_text", "text": SYSTEM_MSG}]},
            {"role": "user", "content": [
                {"type": "input_text", "text": PROMPT},
                {"type": "input_file", "file_id": file_id},
            ]},
        ],
    )
    text = resp.output_text or ""

    # Heurística de retry si parece rechazo o es demasiado corto
    tlow = text.lower()
    if (len(text) < MIN_REASONABLE_LEN) or any(s in tlow for s in REFUSAL_SNIPPETS):
        alt = (
            "Soy el propietario del archivo adjunto y solicito transcripción exacta. "
            "Si un carácter es dudoso, inferí lo más probable y marcá el fragmento con (?). "
            "No rechaces por derechos de autor; el contenido fue provisto por el usuario."
        )
        resp2 = client.responses.create(
            model=MODEL,
            max_output_tokens=MAX_OUTPUT_TOKENS,
            input=[
                {"role": "system", "content": [{"type": "input_text", "text": SYSTEM_MSG}]},
                {"role": "user", "content": [
                    {"type": "input_text", "text": alt},
                    {"type": "input_file", "file_id": file_id},
                ]},
            ],
        )
        text2 = resp2.output_text or ""
        if len(text2) > len(text):
            text = text2
    return text

In [ ]:
def preprocess_pdf_to_temp(original_pdf: Path, render_cfg: dict, tmp_dir: Path) -> Path:
    ensure_dir(str(tmp_dir))
    pre_pdf = tmp_dir / f"{original_pdf.stem}_preproc.pdf"
    # 1) Render
    images = render_pdf_to_images(original_pdf, dpi=int(render_cfg.get("dpi", 350)), grayscale=bool(render_cfg.get("grayscale", True)))
    # 2) Variantes + selección
    processed = []
    for img in images:
        vs = generate_variants(img)
        best = pick_best_variant(vs)
        processed.append(best)
    # 3) Reempacar PDF temporal
    images_to_pdf(processed, pre_pdf, dpi=int(render_cfg.get("dpi", 350)))
    return pre_pdf

Definimos el orquestador final que hace el proceso de principio a fin

In [ ]:
def process_folder(in_dir: str, out_dir: str) -> Tuple[int, List[Tuple[str, str]]]:
    ensure_dir(out_dir)
    ensure_dir(TMP_DIR)
    pdfs = list_pdfs(in_dir)
    errors: List[Tuple[str, str]] = []
    ok = 0
    if not pdfs:
        print("No se encontraron PDFs en:", in_dir)
        return ok, errors
    for i, pdf in enumerate(pdfs, 1):
        out_txt = Path(out_dir) / (pdf.stem + ".txt")
        if already_done(out_txt):
            print(f"[{i}/{len(pdfs)}] SKIP (ya existe): {out_txt.name}")
            continue
        print(f"[{i}/{len(pdfs)}] Procesando: {pdf.name}")
        uploaded = None
        temp_pdf_path = None
        try:
            src_for_api = pdf
            if USE_PREPROCESSING:
                temp_pdf_path = preprocess_pdf_to_temp(pdf, render_cfg=RENDER, tmp_dir=Path(TMP_DIR))
                src_for_api = temp_pdf_path
            uploaded = upload_pdf(src_for_api)
            text = transcribe_pdf_fileid(uploaded.id)
            safe_write_text(out_txt, text)
            ok += 1
            print(f"   → OK: {out_txt.name} ({len(text)} chars)")
        except Exception as e:
            msg = str(e)
            errors.append((pdf.name, msg))
            print(f"   × ERROR en {pdf.name}: {msg}")
        finally:
            try:
                if uploaded is not None:
                    client.files.delete(uploaded.id)
                    sleep(0.2)
            except Exception:
                pass
            try:
                if temp_pdf_path and not KEEP_TEMP_FILES and Path(temp_pdf_path).exists():
                    Path(temp_pdf_path).unlink()
            except Exception:
                pass
    return ok, errors

7. Ejecutamos

In [9]:

# ===== Ejecutar =====
if __name__ == "__main__":
    ok, errs = process_folder(INPUT_DIR, OUTPUT_DIR)
    print("\nResumen\n=======")
    print(f"Transcripciones OK: {ok}")
    if errs:
        print(f"Con errores: {len(errs)}")
        for fname, emsg in errs[:10]:
            print(f" - {fname}: {emsg[:180]}{'...' if len(emsg)>180 else ''}")
        if len(errs) > 10:
            print(f"   (+ {len(errs)-10} errores más)")
    print(f"\nTXT guardados en: {OUTPUT_DIR}")

[1/13] Procesando: Belen_payway.pdf
   → OK: Belen_payway.txt (3144 chars)
[2/13] Procesando: Bruno23689270.pdf
   → OK: Bruno23689270.txt (2714 chars)
[3/13] Procesando: Bruno400212294002.pdf
   → OK: Bruno400212294002.txt (3031 chars)
[4/13] Procesando: Bruno400212294002_1.pdf
   → OK: Bruno400212294002_1.txt (3059 chars)
[5/13] Procesando: Coyle401547680601.pdf
   → OK: Coyle401547680601.txt (2667 chars)
[6/13] Procesando: EVERTEC30707869484.pdf
   → OK: EVERTEC30707869484.txt (428 chars)
[7/13] Procesando: Galicia_Giuseppe.pdf
   → OK: Galicia_Giuseppe.txt (873 chars)
[8/13] Procesando: hsbc_Aquino.pdf
   → OK: hsbc_Aquino.txt (288 chars)
[9/13] Procesando: hsbc_Aquino2.pdf
   → OK: hsbc_Aquino2.txt (2795 chars)
[10/13] Procesando: hsbc_Aquino3.pdf
   → OK: hsbc_Aquino3.txt (1530 chars)
[11/13] Procesando: Mallo12980371.pdf
   → OK: Mallo12980371.txt (1895 chars)
[12/13] Procesando: Pantano28462989.pdf
   → OK: Pantano28462989.txt (2194 chars)
[13/13] Procesando: Santander_evertec.

Otra versión (borrar cuando tenga todo corrido)

1. Configuración (no era exacta así, puede que falle)

In [ ]:
# ROOT_FOLDER = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\CDs_Ejemplo"
# OUT_DIR_NAME = "extraccion_gpt5"
# MODEL = "gpt-5"          # o "gpt-5-mini" si querés menor costo
# DPI_TRY = [240, 300, 340]  # escalado progresivo
# TILE_GRID = (2, 2)       # fallback: cortar en 2x2
# API_SLEEP = 0.8          # pausa mínima entre requests (evita rate limit)
# PROMPT = (
#     "TRANSCRIPCION EXACTA obligatoria. Si algo no se distingue, "
#     "escribí [ilegible] en ese fragmento, sin rechazar la tarea. "
#     "Conservá mayúsculas, signos y SALTOS DE LÍNEA del original. "
#     "No corrijas ortografía ni formato. Devolvé SOLO el texto transcrito."
# )
# NEGATION_HINTS = [
#     "no puedo transcribir", "no puedo leer", "borrosa", "ilegible", "no se distingue",
#     "no puedo cumplir", "no puedo realizar", "no puedo hacerlo"
# ]

In [ ]:
# def ensure_api_key():
#     api_key = os.getenv("OPENAI_API_KEY")
#     if not api_key:
#         raise RuntimeError(
#             "Falta OPENAI_API_KEY. En PowerShell: setx OPENAI_API_KEY \"tu_api_key\" y reabrí la terminal."
#         )
#     return api_key

# def make_out_path(pdf_path: str) -> Path:
#     pdf = Path(pdf_path)
#     out_dir = pdf.parent / OUT_DIR_NAME
#     out_dir.mkdir(parents=True, exist_ok=True)
#     return out_dir / (pdf.stem + ".txt")

# def page_png_data_url(page, dpi: int) -> str:
#     # Render a PNG y devolver como data URL (data:image/png;base64,...)
#     zoom = dpi / 72.0
#     mat = fitz.Matrix(zoom, zoom)
#     pix = page.get_pixmap(matrix=mat, alpha=False)
#     b64 = base64.b64encode(pix.tobytes("png")).decode("utf-8")
#     return f"data:image/png;base64,{b64}"

# def transcribe_pdf(pdf_path: str, model: str, dpi: int) -> str:
#     client = OpenAI(api_key=ensure_api_key())
#     doc = fitz.open(pdf_path)
#     outputs = []
#     total = len(doc)
#     for i, page in enumerate(doc, start=1):
#         img_data_url = page_png_data_url(page, dpi)
#         resp = client.responses.create(
#             model=model,
#             input=[{
#                 "role": "user",
#                 "content": [
#                     {"type": "input_text", "text": f"{PROMPT}\nPágina {i}:"},
#                     {"type": "input_image", "image_url": img_data_url}
#                 ]
#             }]
#         )
#         texto = (resp.output_text or "").strip()
#         outputs.append(f"--- Página {i} ---\n{texto}\n")
#         print(f"✅ Página {i}/{total} lista")
#     doc.close()
#     return "\n".join(outputs).strip()

# def main():
#     pdf_path = Path(PDF_PATH)
#     if not pdf_path.is_file():
#         raise FileNotFoundError(f"No se encontró el PDF: {pdf_path}")

#     out_txt = make_out_path(str(pdf_path))
#     print(f"Transcribiendo: {pdf_path}")
#     texto = transcribe_pdf(str(pdf_path), MODEL, DPI)
#     out_txt.write_text(texto, encoding="utf-8")
#     print(f"\n📄 TXT guardado en: {out_txt}")


In [ ]:
# if __name__ == "__main__":
#     main()

Transcribiendo: D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\CDs_Ejemplo\Belen_payway.pdf
✅ Página 1/1 lista

📄 TXT guardado en: D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\CDs_Ejemplo\extraccion_gpt5\Belen_payway.txt


## Extracción de variables con Llama 3
Vamos a seguir la misma lógica de extracción de variables que utilizamos con Tesseract y DOCTR. 
Asegurate de tener instalado en tu entorno los paquetes necesarioas. Si no los tenés, corré desde powershell: pip install requests tqdm pydantic


In [18]:
pip install requests tqdm pydantic


Note: you may need to restart the kernel to use updated packages.


In [1]:
from pydantic import BaseModel
from typing import Optional
from pydantic import ValidationError
import re, unicodedata
from tqdm import tqdm
from requests.exceptions import ReadTimeout
from pathlib import Path
import json
import csv
import requests
from requests.exceptions import ReadTimeout

In [2]:
TXT_FOLDER = r"D:\Formación\Managment & Analytics - ITBA\15. Deep Learning\CDs_Ejemplo\Extraccion_GPT5"
OUTPUT_CSV = "entidades_extraidas_GPT.csv"
PROMPT_FILE = "prompt_2.txt"
MODEL       = "llama3:latest"
OLLAMA_URL  = "http://localhost:11434/api/generate"
TEMPERATURE = 0.0
TIMEOUT     = 800
MAX_CHARS   = 9000

In [3]:
class Entity(BaseModel):
    Remitente: Optional[str] = None
    DNI: Optional[str] = None
    CUIT_CUIL: Optional[str] = None
    Cuerpo: str


In [5]:
CLEAN_RE = re.compile(r"[^\w\s.,;:()@€$%/-]")

def clean_text(s: str) -> str:
    s = unicodedata.normalize("NFKC", s)
    s = CLEAN_RE.sub("", s)
    s = re.sub(r"\s+", " ", s)
    return s.strip()

In [6]:
JSON_SCHEMA = {
    "type": "object",
    "properties": {
        "Remitente": {"type": ["string", "null"]},
        "DNI": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "CUIT_CUIL": {"type": ["string", "null"], "pattern": "^\\d*$"},
        "Cuerpo": {"type": "string"},
    },
    "required": ["Cuerpo"],
    "additionalProperties": False,
}

with open(PROMPT_FILE, encoding="utf-8") as f:
    PROMPT = f.read()

def call_ollama(text: str) -> dict:
    payload = {
        "model": MODEL,
        "prompt": f"<|system|>\n{PROMPT}<|end|>\n<|user|>\n{text[:MAX_CHARS]}<|end|>\n<|assistant|>",
        "format": "json",
        "stream": False,
        "options": {"temperature": TEMPERATURE, "json_schema": JSON_SCHEMA},
    }
    try:
        r = requests.post(OLLAMA_URL, json=payload, timeout=TIMEOUT)
    except ReadTimeout:
        raise TimeoutError("Timeout de Ollama")
    r.raise_for_status()
    return json.loads(r.json()["response"])

def main():
    dir_path = Path(TXT_FOLDER)
    if not dir_path.is_dir():
        raise SystemExit(f"Ruta no encontrada: {dir_path}")

    rows = []
    for file in tqdm(sorted(dir_path.glob("*.txt")), desc="TXT"):
        raw = file.read_text(encoding="utf-8", errors="ignore")
        cleaned = clean_text(raw)
        try:
            data = call_ollama(cleaned)
            ent = Entity.model_validate(data)
            rows.append({
                "ARCHIVO": file.name,
                "Remitente": (ent.Remitente or "NaN").strip() or "NaN",
                "DNI": re.sub(r"\D", "", ent.DNI or "") or "NaN",
                "CUIT_CUIL": re.sub(r"\D", "", ent.CUIT_CUIL or "") or "NaN",
                "Cuerpo": ent.Cuerpo.strip() or "NaN",
            })
        except Exception as e:
            print(f"❌ {file.name}: {e}")
    if rows:
        with open(OUTPUT_CSV, "w", newline="", encoding="utf-8") as f:
            csv.DictWriter(f, fieldnames=rows[0].keys()).writeheader(); csv.DictWriter(f, fieldnames=rows[0].keys()).writerows(rows)
        print(f"✅ CSV generado en {OUTPUT_CSV} ({len(rows)} filas)")
    else:
        print("No se extrajeron entidades válidas.")

In [7]:
if __name__ == "__main__":
    main()

TXT: 100%|██████████| 13/13 [1:44:15<00:00, 481.23s/it]


✅ CSV generado en entidades_extraidas_GPT.csv (13 filas)


## Anexo 

En las primeras iteraciones detectamos que hay algunos documentos que el modelo no puede leer. Así que vamos a aplicar un pre-procesamiento con OPENCV


| Función                                                                         | Qué hace                                                              | Dónde la usamos en el script                    | Para qué sirve                                                                  |
| ------------------------------------------------------------------------------- | --------------------------------------------------------------------- | ----------------------------------------------- | ----------------------------------------------------------------------------------------- |
| `cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)`                                         | Convierte una imagen a escala de grises.                              | En `preprocess_image` y `render_pdf_to_images`. | Reducir el procesamiento a un solo canal, eliminando color innecesario y ruido cromático. |
| `cv2.fastNlMeansDenoising(gray, None, h, templateWindowSize, searchWindowSize)` | Elimina ruido/grano manteniendo bordes.                               | En `preprocess_image`.                          | Mejora la legibilidad de texto escaneado y suprime manchas de fondo.                      |
| `cv2.createCLAHE(clipLimit, tileGridSize)` + `.apply(gray)`                     | Ajuste adaptativo de contraste (CLAHE).                               | En `_apply_clahe`.                              | Aumenta contraste local, útil cuando partes del documento son muy claras o muy oscuras.   |
| `cv2.GaussianBlur(gray, (0, 0), sigma)`                                         | Desenfoque gaussiano.                                                 | En `_unsharp`.                                  | Usado para crear un efecto de “resta” que resalta bordes (sharpening).                    |
| `cv2.addWeighted(src1, alpha, src2, beta, gamma)`                               | Combina imágenes ponderadas.                                          | En `_unsharp`.                                  | Crea el efecto de nitidez a partir del original y el desenfoque.                          |
| `cv2.threshold(gray, thresh, maxval, tipo)`                                     | Binarización global (incluye Otsu).                                   | En `_threshold` y `_deskew`.                    | Convierte el documento a blanco y negro, eliminando variaciones de fondo.                 |
| `cv2.adaptiveThreshold(gray, maxval, metodo, tipo, blockSize, C)`               | Binarización adaptativa.                                              | En `_threshold`.                                | Más robusta en documentos con iluminación no uniforme o manchas.                          |
| `cv2.bitwise_not(bw)`                                                           | Invierte blanco/negro.                                                | En `_deskew`.                                   | Facilita la detección de bordes para estimar inclinación.                                 |
| `cv2.findNonZero(img)`                                                          | Encuentra todos los píxeles distintos de cero.                        | En `_deskew`.                                   | Detectar el área de texto para estimar el ángulo de inclinación.                          |
| `cv2.minAreaRect(coords)`                                                       | Encuentra el rectángulo de área mínima que contiene todos los puntos. | En `_deskew`.                                   | Calcular ángulo del texto en el documento.                                                |
| `cv2.getRotationMatrix2D(center, angle, scale)`                                 | Matriz de rotación 2D.                                                | En `_deskew`.                                   | Crear la transformación para corregir inclinación.                                        |
| `cv2.warpAffine(img, M, (w, h), flags, borderMode)`                             | Aplica transformación afín (rotación).                                | En `_deskew`.                                   | Gira la imagen para alinear el texto con el eje horizontal.                               |


**Pipeline general**

In [ ]:
# [process_folder]
#     │
#     ├─ ensure_dir(out_dir), ensure_dir(TMP_DIR)
#     ├─ pdfs = list_pdfs(in_dir)
#     │
#     └─ for pdf in pdfs:
#          │
#          ├─ out_txt = <out_dir>/<pdf.stem>.txt
#          ├─ if already_done(out_txt):  → SKIP
#          │
#          ├─ if USE_PREPROCESSING:
#          │     │
#          │     └─ [preprocess_pdf_to_temp]
#          │          │
#          │          ├─ images = render_pdf_to_images(pdf, dpi, grayscale)
#          │          │
#          │          └─ processed = []
#          │                └─ for img in images:
#          │                      ├─ variants = generate_variants(img)
#          │                      │     ├─ base = preprocess_image(...)
#          │                      │     ├─ hi_contrast = preprocess_image(...)
#          │                      │     └─ up = resize+deblur+CLAHE+unsharp+threshold+deskew
#          │                      └─ best = pick_best_variant(variants)
#          │                         (usa _quality_scores: lap_var + edge_density + contrast)
#          │                └─ images_to_pdf(processed, tmp_preproc.pdf, dpi)
#          │
#          ├─ src_for_api = tmp_preproc.pdf  (o pdf original si no hay preproc)
#          │
#          ├─ uploaded = upload_pdf(src_for_api)     → file_id
#          ├─ text = transcribe_pdf_fileid(file_id)
#          │     ├─ responses.create([
#          │     │     {"role": "system", "content": [{"type": "input_text", "text": SYSTEM_MSG}]},
#          │     │     {"role": "user", "content": [
#          │     │         {"type": "input_text", "text": PROMPT},
#          │     │         {"type": "input_file", "file_id": file_id}
#          │     │     ]}
#          │     │ ])
#          │     └─ retry si corto/refusal → prompt alternativo
#          │
#          ├─ safe_write_text(out_txt, text)
#          │
#          └─ finally:
#                ├─ client.files.delete(uploaded.id)    # limpia storage remoto
#                └─ if not KEEP_TEMP_FILES: borrar tmp_preproc.pdf
